In [ ]:
import pandas as pd
import win32com.client

# API를 통한 일별 데이터 조회
def get_daily_price(code):
    # 연결 여부 체크
    objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    bConnect = objCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
    
    stock_chart = win32com.client.Dispatch("CpSysDib.StockChart")
    # 현재가 통신 및 통신 에러 처리 
    rqStatus = stock_chart.GetDibStatus()
    rqRet = stock_chart.GetDibMsg1()
    print("get_daily_price 통신상태", rqStatus, rqRet)
    if rqStatus != 0:
        return False
        
    stock_chart.SetInputValue(0, code)                          # 종목 코드
    stock_chart.SetInputValue(1, ord('2'))                      # 요청 구분 '1': 기간, '2': 개수
    stock_chart.SetInputValue(4, 100)                           # 요청 데이터의 개수
    stock_chart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 13])        # 요청 내용, 0-day, 2-open, 3,-high..
    stock_chart.SetInputValue(6, ord('D'))                      # 분봉 데이터
    stock_chart.SetInputValue(9, '1')                           # 수정 주가
    stock_chart.BlockRequest()

    count = stock_chart.GetHeaderValue(3)

    price_list = []
    for i in range(count):
        day = stock_chart.GetDataValue(0, i)  # 위 SetInputValue(5, [0, 2, 3, 4, 5, 8, 13]) 결과를 순서대로 get
        open = stock_chart.GetDataValue(1, i)
        high = stock_chart.GetDataValue(2, i)
        low = stock_chart.GetDataValue(3, i)
        close = stock_chart.GetDataValue(4, i)
        volume = stock_chart.GetDataValue(5, i)

        price_list.append([day, open, high, low, close, volume])

    labels = ['Day','Open','High','Low','Close','Volume']
    df = pd.DataFrame.from_records(price_list, columns=labels)
    df = df.sort_values(by=['Day'], ascending=True)  # 날짜 기준으로 오름차순 정렬
    #df['Day'] = pd.to_datetime(df['Day'])
    value = df.set_index('Day')
    
    return (value)
    

In [ ]:
# 14일 ATR구하기

def get_atr(raw):
    TR1 = raw['High'] - raw['Low']
    TR2 = abs(raw['Close'].shift(1) - raw['High'])
    TR3 = abs(raw['Close'].shift(1) - raw['Low'])

    df = pd.concat([TR1, TR2, TR3], axis=1)
    tr = df.max(axis=1)
    atr13 = tr.rolling(window=13).mean()
    atr14 = ((atr13.shift(1) * 13 + tr) / 14)
    
    return round(atr14)

In [ ]:
# MA 구하기

def get_ma(raw,day):
    ma = raw['Close'].rolling(window=day).mean()
    
    return round(ma)

In [ ]:
# 주식 현금 매수주문

def buy_code(code, buy_each, buy_price):
    # 연결 여부 체크
    objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    bConnect = objCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False

    # 주문 초기화
    objTrade =  win32com.client.Dispatch("CpTrade.CpTdUtil")
    initCheck = objTrade.TradeInit(0)
    if (initCheck != 0):
        print("주문 초기화 실패")
        return False


    # 주식 매수 주문
    acc = objTrade.AccountNumber[0] #계좌번호
    accFlag = objTrade.GoodsList(acc, 1)  # 주식상품 구분
    print(acc, accFlag[0])
    objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
    objStockOrder.SetInputValue(0, "2")   # 2: 매수
    objStockOrder.SetInputValue(1, acc )   #  계좌번호
    objStockOrder.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
    objStockOrder.SetInputValue(3, code)   # 종목코드
    objStockOrder.SetInputValue(4, buy_each)   # 매수수량
    objStockOrder.SetInputValue(5, buy_price)   # 주문단가
    objStockOrder.SetInputValue(7, "0")   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
    objStockOrder.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

    # 매수 주문 요청
    objStockOrder.BlockRequest()

    rqStatus = objStockOrder.GetDibStatus()
    rqRet = objStockOrder.GetDibMsg1()
    print("buy_code 통신상태", rqStatus, rqRet)
    if rqStatus != 0:
        return False
    
    return True


In [ ]:
# 이동평균과 현재 가격을 비교 매수할 %(비율) 점수 구하기

def get_score(ma, code):
    ma = ma.iloc[-1]
    
    # 연결 여부 체크
    objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    bConnect = objCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False

    # 현재가 객체 구하기
    objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
    objStockMst.SetInputValue(0, code)   #종목 코드 - 코스닥150 레버리지
    objStockMst.BlockRequest()

    # 현재가 통신 및 통신 에러 처리 
    rqStatus = objStockMst.GetDibStatus()
    rqRet = objStockMst.GetDibMsg1()
    print("get_score 통신상태", rqStatus, rqRet)
    if rqStatus != 0:
        return False

    # 현재가 정보 조회
    #code = objStockMst.GetHeaderValue(0)  #종목코드
    #name= objStockMst.GetHeaderValue(1)  # 종목명
    #time= objStockMst.GetHeaderValue(4)  # 시간
    cprice= objStockMst.GetHeaderValue(11) # 종가, 현재가격
    #diff= objStockMst.GetHeaderValue(12)  # 대비
    #open= objStockMst.GetHeaderValue(13)  # 시가
    #high= objStockMst.GetHeaderValue(14)  # 고가
    #low= objStockMst.GetHeaderValue(15)   # 저가
    #offer = objStockMst.GetHeaderValue(16)  #매도호가
    #bid = objStockMst.GetHeaderValue(17)   #매수호가
    #vol= objStockMst.GetHeaderValue(18)   #거래량
    #vol_value= objStockMst.GetHeaderValue(19)  #거래대금
    
    score = 0
    for i in range(6,len(ma)):  # ma3 loc가 6 이고 ma20이 끝 행
        if cprice > ma[i]:
            score += 1
    
    return (score/len(ma))

In [ ]:
# 양봉, 음봉 파악하여 매수 하기

def get_market_trend(data):
    # 연결 여부 체크
    objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    bConnect = objCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False

    # 현재가 객체 구하기
    objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
    objStockMst.SetInputValue(0, 'A233740')   #종목 코드 - 코스닥150 레버리지
    objStockMst.BlockRequest()

    # 현재가 통신 및 통신 에러 처리 
    rqStatus = objStockMst.GetDibStatus()
    rqRet = objStockMst.GetDibMsg1()
    print("get_market_trend 통신상태", rqStatus, rqRet)
    if rqStatus != 0:
        return False

    # 현재가 정보 조회
    #code = objStockMst.GetHeaderValue(0)  #종목코드
    #name= objStockMst.GetHeaderValue(1)  # 종목명
    #time= objStockMst.GetHeaderValue(4)  # 시간
    cprice= objStockMst.GetHeaderValue(11) # 종가, 현재가격
    #diff= objStockMst.GetHeaderValue(12)  # 대비
    open= objStockMst.GetHeaderValue(13)  # 시가
    high= objStockMst.GetHeaderValue(14)  # 고가
    low= objStockMst.GetHeaderValue(15)   # 저가
    #offer = objStockMst.GetHeaderValue(16)  #매도호가
    #bid = objStockMst.GetHeaderValue(17)   #매수호가
    #vol= objStockMst.GetHeaderValue(18)   #거래량
    #vol_value= objStockMst.GetHeaderValue(19)  #거래대금
    
    if cprice > open:  # 양봉이면 매수
        state = "UP"
        
    if cprice < open:  # 음봉이지만 전날 종가보다 상승하고 있으면 매수
        if cprice > data.iloc[-2][3]: # data.iloc[-2][3] -- 전날 close 가격
            state = "UP"
    else:
        state = "DOWN"
        
    return [state, cprice]

In [ ]:
# 현재가 구하기

def get_current_price():
    # 연결 여부 체크
    objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    bConnect = objCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False

    # 현재가 객체 구하기
    objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
    objStockMst.SetInputValue(0, 'A233740')   #종목 코드 - 코스닥150 레버리지
    objStockMst.BlockRequest()

    # 현재가 통신 및 통신 에러 처리 
    rqStatus = objStockMst.GetDibStatus()
    rqRet = objStockMst.GetDibMsg1()
    print("get_current_price 통신상태", rqStatus, rqRet)
    if rqStatus != 0:
        return False

    # 현재가 정보 조회
    #code = objStockMst.GetHeaderValue(0)  #종목코드
    #name= objStockMst.GetHeaderValue(1)  # 종목명
    #time= objStockMst.GetHeaderValue(4)  # 시간
    cprice= objStockMst.GetHeaderValue(11) # 종가, 현재가격
    #diff= objStockMst.GetHeaderValue(12)  # 대비
    #open= objStockMst.GetHeaderValue(13)  # 시가
    #high= objStockMst.GetHeaderValue(14)  # 고가
    #low= objStockMst.GetHeaderValue(15)   # 저가
    #offer = objStockMst.GetHeaderValue(16)  #매도호가
    #bid = objStockMst.GetHeaderValue(17)   #매수호가
    #vol= objStockMst.GetHeaderValue(18)   #거래량
    #vol_value= objStockMst.GetHeaderValue(19)  #거래대금
        
    return cprice

In [ ]:
class BuyError(Exception):
    pass

class get_market_trend_error(Exception):
    pass

class get_score_error(Exception):
    pass

class get_current_price_error(Exception):
    pass

In [ ]:
from time import sleep
import sys

import ChatBotModel

# if __name__ == "__main__":
BUS = ChatBotModel.Bot2ndBUS()  # 텔레그램 봇 기동

# 100일간 일별 데이터 구하기, etf150_data = get_daily_price(code)
code = 'A233740'  # ETF 코스닥150 레버리지

try:
    etf150_data = get_daily_price(code)  # 100일간 데이터 구하기
except Exception as ex:
    print ("크레온 API 연동에 문제가 발생하였습니다",ex)  #telegram bot으로 통보하기
    BUS.sendMessage('크레온 API 연동에 문제가 발생하였습니다')
    sound = firecracker()
    BUS.sendMessage(sound)
    sys.exit()
    
# atr 14일 구하기, atr14 = get_atr(etf150_data)
try:
    atr14 = get_atr(etf150_data)
except Exception as ex:
    print ("크레온 API 연동에 문제가 발생하였습니다",ex)  #telegram bot으로 통보하기
    BUS.sendMessage('크레온 API 연동에 문제가 발생하였습니다')
    sound = firecracker()
    BUS.sendMessage(sound)
    sys.exit()
    
etf150_data.insert(len(etf150_data.columns),"ATR14", atr14)

# ma 일자별 구하기
ma3 = get_ma(etf150_data, 3)  # get_ma("데이터", window값)
ma4 = get_ma(etf150_data, 4)  # get_ma("데이터", window값)
ma5 = get_ma(etf150_data, 5)  # get_ma("데이터", window값)
ma6 = get_ma(etf150_data, 6)  # get_ma("데이터", window값)
ma7 = get_ma(etf150_data, 7)  # get_ma("데이터", window값)
ma8 = get_ma(etf150_data, 8)  # get_ma("데이터", window값)
ma9 = get_ma(etf150_data, 9)  # get_ma("데이터", window값)
ma10 = get_ma(etf150_data, 10)  # get_ma("데이터", window값)
ma11 = get_ma(etf150_data, 11)  # get_ma("데이터", window값)
ma12 = get_ma(etf150_data, 12)  # get_ma("데이터", window값)
ma13 = get_ma(etf150_data, 13)  # get_ma("데이터", window값)
ma14 = get_ma(etf150_data, 14)  # get_ma("데이터", window값)
ma15 = get_ma(etf150_data, 15)  # get_ma("데이터", window값)
ma16 = get_ma(etf150_data, 16)  # get_ma("데이터", window값)
ma17 = get_ma(etf150_data, 17)  # get_ma("데이터", window값)
ma18 = get_ma(etf150_data, 18)  # get_ma("데이터", window값)
ma19 = get_ma(etf150_data, 19)  # get_ma("데이터", window값)
ma20 = get_ma(etf150_data, 20)  # get_ma("데이터", window값)

etf150_data.insert(len(etf150_data.columns),"ma3", ma3)
etf150_data.insert(len(etf150_data.columns),"ma4", ma4)
etf150_data.insert(len(etf150_data.columns),"ma5", ma5)
etf150_data.insert(len(etf150_data.columns),"ma6", ma6)
etf150_data.insert(len(etf150_data.columns),"ma7", ma7)
etf150_data.insert(len(etf150_data.columns),"ma8", ma8)
etf150_data.insert(len(etf150_data.columns),"ma9", ma9)
etf150_data.insert(len(etf150_data.columns),"ma10", ma10)
etf150_data.insert(len(etf150_data.columns),"ma11", ma11)
etf150_data.insert(len(etf150_data.columns),"ma12", ma12)
etf150_data.insert(len(etf150_data.columns),"ma13", ma13)
etf150_data.insert(len(etf150_data.columns),"ma14", ma14)
etf150_data.insert(len(etf150_data.columns),"ma15", ma15)
etf150_data.insert(len(etf150_data.columns),"ma16", ma16)
etf150_data.insert(len(etf150_data.columns),"ma17", ma17)
etf150_data.insert(len(etf150_data.columns),"ma18", ma18)
etf150_data.insert(len(etf150_data.columns),"ma19", ma19)
etf150_data.insert(len(etf150_data.columns),"ma20", ma20)

################################################################################

# 총자산 대비 Rsik Management
one_trading_risk = 2  # 2%
equity = 1000000  # 자본금 백만원
one_trading_risk_price = equity * one_trading_risk / 100 
each = 1  # 종목수
etf150_data = etf150_data.reset_index()
ATR = etf150_data.loc[len(etf150_data)-1,'ATR14']

print (ATR)
print (current_price)
print (stop_loss_price)
print (one_trading_risk_price_each)
print (buy_each)
print (purchase_amount)

In [ ]:
# 총5회 매수, 3시 10분, 15분, 20분, 25분, 28분

try:
    #현재가 구하기, current_price = get_current_price()
    ret = get_current_price()
    if ret == False:
        raise get_current_price_error()
    else:
        current_price = ret
       
    # 현재가격 기준으로 score 구하기, score = get_score(etf150_data, code)
    ret = get_score(etf150_data, code)
    if ret == False:
        raise get_score_error()
    else:
        score = ret
    

    # RISK MANAGEMENT
    stop_loss_price = current_price - (ATR * 2)  # 2N으로 계산
    one_trading_risk_price_each = one_trading_risk_price / each
    buy_each = int(round(one_trading_risk_price_each / (ATR * 2)))  # 2N으로 계산
    purchase_amount = buy_each * current_price
    
    # 매수조건 구하기, 리턴할때 UP/DOWN과 현재가 돌려줌 entry = get_market_trend(etf150_data)
    ret = get_market_trend(etf150_data)  # return값으로 [0] - UP/Down, [1] - 현재가
    if ret == False:
        raise get_market_trend_error()
    else:
        entry = ret
    
    # 현금매수하기, buy_code(code, buy_each, buy_price)
    if entry[0] == "UP":
        ret = buy_code(code, buy_each, entry[1])
        if ret == False:
            raise BuyError()
        else:
            BUS.sendMessage('크레온 API를 통해 매수하였습니다.')
            BUS.sendPhoto(open('img.jpg', 'rb'))  # 매매 현황을 이미지 파일로 전송

except get_market_trend_error:
    BUS.sendMessage('get_market_trend 크레온 API 연동에 문제가 발생하였습니다.')
    sys.exit()
except get_score_error:
    BUS.sendMessage('get_score 크레온 API 연동에 문제가 발생하였습니다.')
    sys.exit()
except get_current_price_error:
    BUS.sendMessage('get_current_price 크레온 API 연동에 문제가 발생하였습니다.')
    sys.exit()
except BuyError:
    print ("주식 매수시 크레온 API 연동에 문제가 발생하였습니다")  #telegram bot으로 통보하기
    BUS.sendMessage('주식 매수시 크레온 API 연동에 문제가 발생하였습니다.')
    sys.exit()
except Exception as ex:
    print ('CREON API ERROR', ex)
    BUS.sendMessage('CREON API ERROR')
    sys.exit()

# 투자자금 * ((총자산 Risk Management 2% / ATR) x 장세 점수) = 매수금액
# 양봉시 매수, 전일종가대비 상승시 음봉일지라도 매수, 음봉시 매도